    ... looking at a classic FACTORY PATTERN, heavily mixed with DELAYED
    EXECUTION (Lazy Initialisation).

    Mapping this to how you would architect things in Kotlin is actually the
    perfect way to understand Isaac Lab's underlying design philosophy.

    Here is exactly how `func` acts as a factory in this architecture:


---
1. THE BLUEPRINT (The Data Class)
    When you write `cgf_cone = sim_utils.ConeCfg(...)`, you are NOT creating a
    3D cone in the world. You are essentially just instantiating a standard 
    Kotlin `data class`.

    It holds all the configuration variables (radius, height, materials) in a
    lightweight object, completely isolated from the heavy C++ physics engine.


2. THE FACTORY METHOD (`.func`)
    In Isaac Lab, every configuration object has a property called `func`. This
    property is literally a pointer to the specific Python function (the factory
    worker) that knows how to parse that exact blueprint and talk to the
    Omniverse engine to build it.

    If you were writing this in Kotlin, it would look very similar to a
    Companion Object factory or a functional interface:

```Kotlin
// The Kotlin equivalent of what Isaac Lab is doing
val coneBlueprint = ConeCfg(radius = 0.15, height = 0.5)

// Calling the factory method to physically build it at a specific path
coneBlueprint.func.invoke("/World/Objects/Cone1", coneBlueprint)
```


---
WHY DO THEY DO IT THIS WAY?
    Isaac Lab forces you to use this factory pattern because of PARALLEL AI
    TRAINING.

    When you train Reinforcement Learning robots, you don't just spawn one 
    environment; you spawn 4096 identical environments simultaneously. By 
    passing the blueprint (`ConeCfg`) from the execution (`.func`), you can
    define your robot once, pass that single lightweight configuration object
    to a massive cloner function, and have the cloner rapidly fire `.func` 4096
    times across your 5090's vRAM.  

---

In [ ]:
o               .        ___---___                    .
       .              .--\        --.     .     .         .
                    ./.;_.\     __/~ \.
                   /;  / `-'  __\    . \
 .        .       / ,--'     / .   .;   \        |
                 | .|       /       __   |      -O-       .
                |__/    __ |  . ;   \ | . |      |
                |      /  \\_    . ;| \___|
   .    o       |      \  .~\\___,--'     |           .
                 |     | . ; ~~~~\_    __|
    |             \    \   .  .  ; \  /_/   .
   -O-        .    \   /         . |  ~/                  .
    |    .          ~\ \   .      /  /~          o
  .                   ~--___ ; ___--~
                 .          ---         .              -JT

    ... we move away from just placing static objects in a scene and start
    dealing with RIGID OBJECT--entities that actually obey the laws of physics,
    have amss, and collide with things.

    In the context of your Lego-assembling SO-101 arm project, this is 
    foundational. A Lego brick dropping onto a table or being pushed by a 
    gripper is a "Rigid Object." If you don't master how the physics engine 
    tracks and updates their states, your robot will just phase through the
    bricks.

    ...


---
1. THE NEW DEPENDENCIES: WHY DO WE NEED THEM?
    To handle physics, we introduce four new imports:

    - `torch`: Used for high-performance tensor computations. Instead of 
      iterating through objects one by one in Python (which is very slow), Isaac
      Sim uses PyTorch to calculate the positions, velocities, and states of
      thousands of objects simultaneously on the GPU.
    - `math_utils`: Provides mathematical utilities specifically for simulation,
      like randomising the spawn positions of objects.
    - `RigidObjectCfg`: The configuration class. This is where you define the
      "blueprint" of the object (its mass, its 3D mesh, its initial spawn state)
    - `RigidObject`: The actual active class that represents the spawned object
      in the simulation, managing its physical properties and state.



2. `design_scene()`: THE NEW WAY TO SPAWN OBJECTS
    In previous tutorials, you used basic `func` methods to spawn the ground 
    plane and lights. While great for simple scenes, it is terrible for complex
    setups (like a bin full of 500 lego bricks) because it is repetitive.

THE NEW APPROACH
    Instead of spawning objects manually, we wrap the spawn configuration inside
    `RigidObjectCfg`

    - HOW IT WORKS BY EXAMPLE: First, the code creates multiple `XForm` prims
      (empty anchor points in 3D space) named `world/origin_0`, `world/origin1`,
      etc.
    - THE REGEX MAGIC: When you pass the configuration to the `RigidObject` 
      class, you give it a regular expression  path (like `world/origin.*`). The
      engine automatically finds every matching anchor point and spawns a child
      object (a Cone in this video) under it automatically.
    - THE RESULT: You instantly get `world/origin_0/cone`, `world/origin_1/cone`
      , etc., all with their physics handles already initialised. The initial
      state defaults to the "identity pose" (Position 0, Rotation 0, Velocity 0)



3. `run_simulation()`: THE PHYSICS LOOP
    This function is the heartbeat of your simulation. it runs at a time step of
    0.017 seconds (roughly 60 Hz).

    Inside this loop, the video introduces a mechanism to reset the cones every
    250 steps so they drop again. To do this correctly, you must understand a 
    critical concept: THE WORLD FRAME. 


THE "WORLD FRAME" TRAP:
    When you want to teleport an object (like resetting a Lego piece to the top
    of the workspace), you cannot just give it a relative local positon based on
    its parent anchor. The physics engine operates strictly in the GLOBAL 
    COORDINATE SYSTEM (WORLD FRAME).

    - THE FIX: You must calculate the object's exact global position and 
      movement, then apply a random offset (using `sample_cylinder`) so the 
      cones don't drop in the exact same spot every time. 



4. THE 5-STEP UPDATE CYCLE
    To actually apply these resets and make the simulation run, you must follow
    a very specific order of operations inside the loop. If you mess this order
    up when training your RL agent later, your simulation will break or return
    garbage data:

    1. `write_root_state_to_sim()`: You take the new, randomised global 
       positions






---

--- Prims (short for Primitives) in Isaac Sim are the fundamental building 
    blocks of a USD (Universal Scene Description) scene, acting as nodes in a
    hierarchical scene graph. They represent individual objects or entities
    --such as meshes, lights, cameras, or robots--and define their properties, 
    transofrmations, and relations within the simulation.

    Key Details about prims in Isaac Sim:
    - STRUCTURE: Prims are organised hierarchically, similar to a file directory
      structure (e.g., `/World/Robot/arm`).
    - TYPES: Common types include `Xform` (for transformation/grouping), `Mesh`
      (for geometry), `Light`, and `Camera`.
    - ATTRIBUTES: These define the data of a prim, such as position, rotation,
      scale, color, or physical properties like mass and friction.#
    - RELATIONSHIPS: Prims can be linked to others, such as connecting a mesh 
      prim to a material prim for rendering .
    - USAGE IN ISAAC LAB: Isaac Lab uses configurations to spawn and manage 
      these prims, allowing for the creation of complex simulation environments.
    - PHYSICS: Prims can be made physical (e.g., rigid bodies, colliders) to 
      interact with the environment, or remain static.   



--- In Isaac Sim (and OpenUSD), an Xform (Transform) is a specialised type of
    prim used to store and manage spatial transformation data--specifically
    translation, rotation, and scaling--for its child prims. They are 
    fundamental for organising scene hierarchies, grouping objects, and 
    animating robots, as transformations applied to a parent Xform propagate
    to all its children.

    KEY ASPECTS of Xforms in Isaac Sim:
    - HIERARCHY MANAGEMENT: Xforms act as containers (folders) for objects. 
      Moving a parent Xform moves everything inside it, making them essential 
      for assembling complex robots of props.
    - TRANSFORM OPERATIONS: They store data via `xformOp:translate`, 
      `xformOp:rotate` anhd `xformOp:scale` to define local positioning.
    - REFERENCE vs. PAYLOAD; In the stage tree, references (lightweight) are
      indicated by an orange arrow, while payloads (active, modifiable data) are
      indicated by a blue arrow on the Xform icon.)
    - VISUALISATION: `Xform Visualization Extension` can be used to make Xforms
      selectable in the viewport.
    - PHYSICS: When applying rigid body properties, they are typically applied 
      to Xforms that contain visual meshes.    